In [10]:
import requests
import pandas as pd
import datetime
from datetime import datetime, timedelta
import pytz
import seaborn as sns
import numpy as np

In [14]:
url = "https://aerodatabox.p.rapidapi.com/airports/iata/DME"

headers = {
	"X-RapidAPI-Key": "90699f41ebmsh8a8aef972097524p1bc434jsn473111efa6f3",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"icao":"UUDD","iata":"DME","localCode":"ДМД","shortName":"Domodedovo","fullName":"Moscow, Domodedovo","municipalityName":"Moscow","location":{"lat":55.4088,"lon":37.9063},"country":{"code":"RU","name":"Russia"},"continent":{"code":"EU","name":"Europe"},"timeZone":"Europe/Moscow","urls":{"webSite":"http://www.domodedovo.ru/en/","wikipedia":"https://en.wikipedia.org/wiki/Domodedovo_International_Airport","twitter":"http://twitter.com/AirportDme","googleMaps":"https://www.google.com/maps/@55.408798,37.906299,14z","flightRadar":"https://www.flightradar24.com/55.41,37.91/14"}}


In [17]:
response.json()['iata']

'DME'

In [13]:
def icao_airport_codes(latitudes, longitudes):

    #assert len(latitudes) == len(longitudes)
  
    list_for_df = []

    for i in range(len(latitudes)):

        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/70/16"

        querystring = {"withFlightInfoOnly":"true"}

        headers = {
          "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
          "X-RapidAPI-Key": "3c16ed817amshbef86ec42129e0bp1cefefjsn62c8e9534fe6"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)

        list_for_df.append(pd.json_normalize(response.json()['items']))
    
    icao_airport_codes = pd.concat(list_for_df, ignore_index=True)
    icao_airport_codes.loc[icao_airport_codes['icao'] == 'EDDF', 'city_id'] = '1'
    icao_airport_codes.loc[icao_airport_codes['icao'] == 'EDFM', 'city_id'] = '1'
    icao_airport_codes.loc[icao_airport_codes['icao'] == 'EDDB', 'city_id'] = '2'
    icao_airport_codes.loc[icao_airport_codes['icao'] == 'EDDK', 'city_id'] = '3'
    icao_airport_codes.loc[icao_airport_codes['icao'] == 'EHBK', 'city_id'] = '3'  
    icao_airport_codes.loc[icao_airport_codes['icao'] == 'EDDM', 'city_id'] = '4'  
    icao_airport_codes.loc[icao_airport_codes['icao'] == 'EDDH', 'city_id'] = '5'  
    
    icao_airport_codes = icao_airport_codes.rename(columns={'municipalityName': 'municipality_name', 
                                          'name': 'city_name',
                                          'countryCode': 'country', 
                                          'location.lat': 'latitude', 
                                          'location.lon': 'longitude',
                                          'icao': 'icao_code',
                                          'iata': 'iata_code',
                                          })
    
    
    
    icao_airport = icao_airport_codes[['city_id','icao_code','iata_code','iata_code','country','municipality_name','latitude','longitude']]
    return icao_airport


la = [50.0638, 52.3112, 50.5611, 48.0815, 53.3300]
lo = [8.4056, 13.2418, 6.5710, 11.3430, 10.0000]

#la = [50.0638]
#lo = [8.4056]

icao_airport = icao_airport_codes(la,lo) 
icao_airport

,city_id,icao_code,iata_code,iata_code,country,municipality_name,latitude,longitude
0,1,EDDF,FRA,FRA,DE,Frankfurt-am-Main,50.026400,8.543129
1,1,EDFM,MHG,MHG,DE,Mannheim,49.473057,8.514166
2,2,EDDB,BER,BER,DE,Berlin,52.351390,13.493889
3,3,EDDK,CGN,CGN,DE,Cologne,50.865900,7.142739
4,3,EHBK,MST,MST,NL,Maastricht,50.911700,5.770140
5,4,EDDM,MUC,MUC,DE,Munich,48.353800,11.786100
6,5,EDDH,HAM,HAM,DE,Hamburg,53.630400,9.988229


In [15]:
#arrivals_c['Departure Airport'].value_counts().nlargest(50)

In [11]:
arrivals_c[arrivals_c['Aircraft Model']=='Airbus A380-800']

,Arrival Airport,Flight Number,Airline Name,Arrival Time,Arrival Terminal,Departure Airport,Aircraft Model,Data Retrieval Time
218,Munich,EK 49,Emirates,06-03-2023 12:30,1C,Dubai City,Airbus A380-800,06-03-2023 08:45
371,Munich,EK 51,Emirates,06-03-2023 19:35,1C,Dubai City,Airbus A380-800,06-03-2023 08:45
526,Frankfurt-am-Main,SQ 25,Singapore,06-03-2023 09:50,1,New York,Airbus A380-800,06-03-2023 08:45
1419,Frankfurt-am-Main,EK 47,Emirates,06-03-2023 19:25,2,Dubai City,Airbus A380-800,06-03-2023 08:45


In [60]:
#cities = ['EDDB','EGCC','EDDH','EGLC']      
#cities = ['EDDB','EDDM','EDDF'] 
cities = ['EDDB']

def flight_arrivals(cities):
    arrivals = {'arrival_airport':[],
                'flight_number':[],
                'airline_name':[],
                'arrival_time':[],
                'arrival_terminal':[],
                'departure_airport':[],               
                'aircraft_model':[],
                'data_retrieval_time':[]
               }
    
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)
    current_date_time = (now.strftime('%Y-%m-%dT%H:%M'))
    current_date_time_table = (now.strftime('%d-%m-%Y %H:%M'))
    end_time_cal = datetime.now() + timedelta(hours=12)
    end_date_time = end_time_cal.strftime('%Y-%m-%dT%H:%M')
    
    for city in cities:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{city}/{current_date_time}/{end_date_time}"
        querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"true","withLocation":"false"}

        headers = {
                    "X-RapidAPI-Key": "3c16ed817amshbef86ec42129e0bp1cefefjsn62c8e9534fe6",
                    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                   }

        response = requests.request("GET", url, headers=headers, params=querystring)
        
        ###########
        
        url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{city}"

        headers = {
                    "X-RapidAPI-Key": "3c16ed817amshbef86ec42129e0bp1cefefjsn62c8e9534fe6",
                    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

        response1 = requests.request("GET", url, headers=headers)

        
        for i in response.json()['arrivals']:
            arrivals['departure_airport'].append(i['departure']['airport']['name'])
            arrivals['airline_name'].append(i['airline']['name'])
            arrivals['flight_number'].append(i['number'])
            arrivals['arrival_time'].append((datetime.strptime((i['arrival']['scheduledTimeLocal'].split('+')[0]),'%Y-%m-%d %H:%M')).strftime('%d-%m-%Y %H:%M'))
            try:
                arrivals['arrival_terminal'].append(i['arrival']['terminal'])
            except:
                arrivals['arrival_terminal'].append('-')
                
            try:
                arrivals['aircraft_model'].append(i['aircraft']['model'])
            except:
                arrivals['aircraft_model'].append('-')
        
            arrivals['arrival_airport'].append(response1.json()['municipalityName'])
            arrivals['data_retrieval_time'].append(current_date_time_table)
            
            

    arrivals_df = pd.DataFrame.from_dict(arrivals)
    
    arrivals = arrivals_df.rename(columns={'arrival_airport': 'Arrival Airport', 
                                          'flight_number': 'Flight Number',
                                          'airline_name': 'Airline Name', 
                                          'arrival_time': 'Arrival Time', 
                                          'arrival_terminal': 'Arrival Terminal',
                                          'departure_airport': 'Departure Airport',
                                          'aircraft_model': 'Aircraft Model',
                                           'data_retrieval_time': 'Data Retrieval Time'
                                          })
    #arrivals['Arrival Time']= pd.to_datetime(arrivals['Arrival Time'])
    #arrivals['Data Retrieval Time']= pd.to_datetime(arrivals['Data Retrieval Time'])
    
    return arrivals
        
        
       
#flight_arrivals(cities)
arrivals = flight_arrivals(cities)

In [61]:
arrivals_c = arrivals.copy()

arrivals_c[arrivals_c['Aircraft Model']=='Airbus A380-800']

,Arrival Airport,Flight Number,Airline Name,Arrival Time,Arrival Terminal,Departure Airport,Aircraft Model,Data Retrieval Time
144,Manchester,EK 17,Emirates,06-03-2023 11:15,1,Dubai City,Airbus A380-800,06-03-2023 10:54
216,Manchester,EK 19,Emirates,06-03-2023 18:40,1,Dubai City,Airbus A380-800,06-03-2023 10:54
